# Procesado Espectros Raman NANOBASE

En este *Notebook* se realiza un preprocesado básico a los espectros Raman medidos con Nanobase. 

Los datos obtenidos de NanoBase corresponden a espectros en bruto a los que se les ha restado el *background*. El programa de medición no realiza ningun otro tipo de tratamiento a los datos. Por ello es necesario realizar diferentes pasos previos al análisis de los espectros:

1. Whittaker Smoother: Método de suavizado aplicado a señales con la finalidad de reducir el ruido, conservando los picos caracteristicos del material. El método funciona minimizando la suma de las diferencias al cuadrado entre cada punto de datos y su estimación suavizada, sujeto a una restricción de que la segunda diferencia de los valores suavizados es pequeña. Esta restricción tiene el efecto de penalizar las variaciones en la tasa de cambio de los valores suavizados, resultando en una señal más suave.

2. Baseline: 

Al contrario que en el preprocesado de la Sonda Raman de 532nm, no es necesario corregir el Background ya que el propio software de medida se encarga de hacerlo.


In [2]:
import numpy as np
import pandas as pd
#%matplotlib inline
import re
import os
import math
import numpy as np
import pandas as pd
import joblib
import warnings
import time
import shutil
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, Normalizer
from scipy.linalg import solveh_banded  # para hacer ALS
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.signal import argrelextrema
from itertools import product
from scipy.signal import savgol_filter
from functools import reduce

# Directorio en el que se encuentran guardados los datos de Raman
path_nanobase_mac = '/Users/drea/Library/CloudStorage/OneDrive-UNICAN/RamanNanoBase/CONCHAS_ARQ/CSV'
path_nanobase_windows ='D:\OneDrive - UNICAN\RamanNanoBase\CONCHAS_ARQ\CSV'
# Elegimos la ruta según el sistema operativo
path_nanobase = path_nanobase_windows
# Obtén la lista de archivos en la carpeta
files = os.listdir(path_nanobase)
# Filtrar solo los archivos CSV
csv_files = [f for f in files if f.endswith('.CSV')]

print("Número de archivos CSV: ", len(csv_files))

# Copiamos los archivos a la carpeta contenedora. Esto se realiza para poder actualizar las nuevas medidas
def copiar_archivos(origen, destino, archivos):
    # Copiar cada archivo seleccionado a la carpeta de destino si no existe
    for archivo in archivos:
        ruta_origen = os.path.join(origen, archivo)
        ruta_destino = os.path.join(destino, archivo)
        if os.path.exists(ruta_origen):
            shutil.copy2(ruta_origen, ruta_destino)
            print(f"Archivo {archivo} copiado correctamente.")
        else:
            print(f"El archivo {archivo} no existe en la carpeta de origen.")

# Ejemplo de uso
carpeta_origen = 'D:\OneDrive - UNICAN\RamanNanoBase\CONCHAS_ARQ\CSV'
carpeta_destino = "D:\OneDrive - UNICAN\Escritorio\Conchas\CSV_nanobase"

# copiar_archivos(carpeta_origen, carpeta_destino, csv_files)

Número de archivos CSV:  9


In [5]:
# Directorios de medida
csv_files = os.listdir(carpeta_destino)
print("Número de archivos CSV: ", len(csv_files))

Número de archivos CSV:  9


In [6]:
class WhittakerSmoother(object):
    def __init__(self, signal, smoothness_param, deriv_order=1):
        self.y = signal
        assert deriv_order > 0, 'deriv_order must be an int > 0'
        # Compute the fixed derivative of identity (D).
        d = np.zeros(deriv_order*2 + 1, dtype=int)
        d[deriv_order] = 1
        d = np.diff(d, n=
        deriv_order)
        n = self.y.shape[0]
        k = len(d)
        s = float(smoothness_param)
        # Here be dragons: essentially we're faking a big banded matrix D,
        # doing s * D.T.dot(D) with it, then taking the upper triangular bands.
        diag_sums = np.vstack([
            np.pad(s*np.cumsum(d[-i:]*d[:i]), ((k-i,0),), 'constant')
            for i in range(1, k+1)])
        upper_bands = np.tile(diag_sums[:,-1:], n)
        upper_bands[:,:k] = diag_sums
        for i,ds in enumerate(diag_sums):
            upper_bands[i,-i-1:] = ds[::-1][:i+1]
        self.upper_bands = upper_bands
    def smooth(self, w):
        foo = self.upper_bands.copy()
        foo[-1] += w  # last row is the diagonal
        return solveh_banded(foo, w * self.y, overwrite_ab=True, overwrite_b=True)

def als_baseline(intensities, asymmetry_param=0.0001, smoothness_param=1e4,
                 max_iters=20, conv_thresh=1e-6, verbose=False):
    """
    Applies the asymmetric least squares (ALS) method to fit the baseline.

    Parameters:
    - intensities (numpy.ndarray or pandas.Series): Numpy array or pandas Series representing the Raman intensities.
    - asymmetry_param (float): Asymmetry parameter for the fit.
    - smoothness_param (float): Smoothness parameter for the fit.
    - max_iters (int): Maximum number of iterations.
    - conv_thresh (float): Convergence threshold.
    - verbose (bool): Flag to print debugging information.

    Returns:
    - baseline (numpy.ndarray): Numpy array representing the fitted baseline.
    """
    if isinstance(intensities, pd.Series):
        intensities = intensities.values

    smoother = WhittakerSmoother(intensities, smoothness_param, deriv_order=2)
    p = asymmetry_param
    w = np.ones(intensities.shape[0])

    for i in range(max_iters):
        z = smoother.smooth(w)
        mask = intensities > z
        new_w = p * mask + (1 - p) * (~mask)
        conv = np.linalg.norm(new_w - w)
        if verbose:
            print(i + 1, conv)
        if conv < conv_thresh:
            break
        w = new_w
    else:
        None
        #print('ALS did not converge in %d iterations' % max_iters)
    return z

def subtract(df, df_background):
    df_subtracted = df.sub(df_background.iloc[0], axis=1)
    return df_subtracted

def find_baseline(df, asymmetry_param=0.0001, smoothness_param=1e4,
                 max_iters=20, conv_thresh=1e-6):
    """
    Apply baseline correction to each row in a DataFrame.
    Input:
        df: DataFrame, contains the spectral data.
    Output:
        DataFrame, with baseline corrected for each spectrum.
    """
    D0_BL = df.copy()
    for i in range(len(df)):
        # Apply ALS baseline correction to each row
        bl = als_baseline(df.iloc[i, :].values, asymmetry_param, smoothness_param,
                 max_iters, conv_thresh)
        D0_BL.iloc[i, :] = bl
    return D0_BL

def correct_baseline(data, baseline):
    """
    Corrects the baseline of the data.
    Parameters:
    - data (pandas.DataFrame or pandas.Series): DataFrame or Series containing the data.
    - baseline (pandas.DataFrame or pandas.Series): DataFrame or Series with the baselines.
    Returns:
    - corrected_data (pandas.DataFrame): DataFrame with the corrected data.
    """
    # Ensure that both data and baseline are DataFrames
    if not isinstance(data, pd.DataFrame):
         data = pd.DataFrame(data)
    if not isinstance(baseline, pd.DataFrame):
         baseline = pd.DataFrame(baseline)
    # Perform baseline correction
    
    corrected_data = data - baseline

    return corrected_data


def normalize_spectra(X_filtered):
    """
    Preprocessing of spectral data.
    Input:
        X_filtered: DataFrame, raw spectral data.
    Output:
        DataFrame, preprocessed spectral data.
    """
    X_norm_list = []
    for _, row in X_filtered.iterrows():
        # Normalize each spectrum by dividing by the sum and multiplying by a constant
        total = np.sum(row)
        X_norm_list.append(pd.DataFrame([row / total * 2000]))
    X_norm = pd.concat(X_norm_list, ignore_index=True)
    # Apply Savitzky-Golay filter for smoothing
    X_norm_flt = savgol_filter(X_norm, 21, 2)
    X_norm_flt = pd.DataFrame(X_norm_flt, columns=X_filtered.columns)
    '''# Perform baseline correction
    bl = find_baseline(X_norm_flt)
    X_norm_flt = X_norm_flt - bl'''
    # Standardize the data using StandardScaler
    scaler = StandardScaler()
    z_numpy = X_norm_flt.values
    z_numpy_scaled_rows = scaler.fit_transform(z_numpy.T).T
    X_norm_flt_stdz = pd.DataFrame(data=z_numpy_scaled_rows, columns=X_filtered.columns)
    return X_norm_flt_stdz

def SNV(input_data):
    """
    Procesado de datos: SNV (Standard Normal Variate).
    1) Media de cada channel --> axis=1 (todas las columnas).
    2) Se le resta su media a cada "channel" --> axis=0 (todas las filas).
    3) Divide cada channel por su StDev --> axis=1 (todas las columnas).
    :parameter *input_data*: matriz de datos, en formato: "channels(f) x time(c)".
    :return: matriz de datos, una vez normalizados
    """
    data_12 = input_data.sub(input_data.mean(axis=1), axis=0)
    data_snv = data_12.div(input_data.std(axis=1), axis=0)
    return data_snv

def cut_spectrum(df, min_wavelength, max_wavelength):
    """
    Cuts the spectrum to a specific range.
    Parameters:
    - df (pandas.DataFrame): DataFrame containing the spectrum.
    - min_wavelength (int): Minimum wavelength.
    - max_wavelength (int): Maximum wavelength.
    Returns:
    - df_cut (pandas.DataFrame): DataFrame containing the spectrum within the specified range.
    """
    df.columns = df.columns.astype(float) #si tus columnas deben ser números flotantes
    df_cut = df.loc[:, (df.columns >= min_wavelength) & (df.columns <= max_wavelength)]
    return df_cut

In [15]:
csv_files

['160424_LIT4_1000ms_n5_potenica1c_mapa_abajo.CSV',
 '170424_LIT2_1000ms_n5_potencia1c_mapa_abajo.CSV',
 '180424_LIT1_1000ms_n5_potencia1c_mapa_abajo.CSV',
 '180424_LIT3_1000ms_n5_potencia1c_mapa_abajo.CSV',
 '190424_LIT5_1000ms_n10_potencia1c_abajo.CSV',
 '230424_LIT6_1000ms_n5_potencia1c_abajo.CSV',
 '240424_LIT235.3_2000ms_n10_potencia1c_abajo.CSV',
 '250424_LIT885.1_3000ms_n8_potencia1c_abajo.CSV',
 '290424_LIT921.1_1500ms_n8_potencia1c_abajo.CSV']

In [17]:
def create_df_raw(folder_path, csv_files , cut= 'cut'):
    dataframes = []
    lista_archivos = []
    lista_muestras = []

    path = folder_path
    for csv_file in csv_files:
            csv_file_complete = os.path.join(path, csv_file)
            df_pre = pd.read_csv(csv_file_complete, engine='python', sep=',', skiprows=14, decimal='.')
            resultados = []
            archivo= csv_file

            # # Drop unwanted column
            if 'Unnamed: 1937' in df_pre.columns:
                df_pre = df_pre.drop(columns=['Unnamed: 1937'])

            # Realizamos el preprocesado de los datos
            # df_pre_plot = df_pre.copy()
            # if cut == 'cut':
            #     df_cut = cut_spectrum(df_pre, min_wavelength=179, max_wavelength=2800)
            # else:
            #     df_cut = df_pre.copy()
            # df_find_baseline = find_baseline(df_cut)
            # df_media_corrected = correct_baseline(df_cut, df_find_baseline)


            # Podemos plotear el paso a paso para comprobar que se ha realizado correctamente
            # df_pre_plot.columns = df_pre.columns.astype(float)
            # df_pre_plot.mean(axis=0).plot()
            # df_cut.mean(axis=0).plot()
            # df_find_baseline.mean(axis=0).plot()
            # df_media_corrected.mean(axis=0).plot()
            print(archivo)

            # df = df_media_corrected.copy()

            # nombre_archivo = archivo # Nombre del archivo
            # nombre_muestra = nombre_archivo.split("_")[3] # Nombre de la muestra
            # fecha_medida = nombre_archivo.split("_")[0].split("-")[0]  # Fecha de la medida
            # hora_medida = nombre_archivo.split("_")[0].split("-")[1]    # Hora de la medida
            # tipo_laser =   "Nanobase"
            # tipo = 'raw'
            # df.index = pd.MultiIndex.from_tuples([( tipo_laser, nombre_muestra, tipo,
            #                                             fecha_medida,hora_medida, nombre_archivo, X, Y, ) for X, Y in df.index],
            #                                       names=[ "Instrumento","Muestra","Tipo",  "Fecha","Hora", "Archivo","X", "Y"])

            # Guardamos el nombre de los archivos y de las muestras en listas
            # lista_archivos.append(nombre_archivo)
            # lista_muestras.append(nombre_muestra)
            # # Agrega el DataFrame a la lista
            dataframes.append(df_pre)

    # Concatena todos los DataFrames en uno solo
    data = pd.concat(dataframes)
    return data

data = create_df_raw(carpeta_destino, ['160424_LIT4_1000ms_n5_potenica1c_mapa_abajo.CSV'], cut='cut')
data

160424_LIT4_1000ms_n5_potenica1c_mapa_abajo.CSV


,X,Y,Z,XINDEX,YINDEX,ZINDEX,-77.043,-74.805,-72.569,-70.332,...,3304.067,3305.232,3306.397,3307.567,3308.734,3309.900,3311.064,3312.227,3313.389,3314.551
0,-187.014249,-51.936528,0,0,0,0,316.4,338.6,326.8,320.0,...,4474.6,4404.8,4346.4,4612.4,4417.6,4518.0,4493.4,4438.8,4383.6,4475.2
1,-183.514249,-51.936528,0,1,0,0,312.8,299.8,312.8,321.6,...,4322.4,4244.4,4257.8,4252.6,4138.2,4296.6,4187.4,4315.8,4197.0,4162.8
2,-180.014249,-51.936528,0,2,0,0,328.0,339.4,319.2,336.0,...,5390.4,5342.6,5404.0,5395.6,5353.6,5278.6,5324.4,5412.8,5258.0,5287.8
3,-176.514249,-51.936528,0,3,0,0,326.4,290.6,310.0,309.6,...,3448.4,3580.8,3538.8,3577.6,3466.8,3647.6,3439.2,3559.0,3518.6,3496.0
4,-173.014249,-51.936528,0,4,0,0,306.4,313.4,340.4,319.6,...,4960.2,4862.8,4820.2,4977.6,4862.6,4931.0,4844.8,4894.2,4775.6,4816.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4489,169.985751,50.563472,0,102,41,0,319.6,313.0,354.4,318.4,...,3613.8,3693.0,3751.2,3793.4,3620.0,3666.0,3663.8,3665.2,3662.8,3593.2
4490,173.485751,50.563472,0,103,41,0,334.8,333.8,336.4,299.6,...,3157.6,3116.4,3076.4,3080.6,3110.0,3130.6,3135.0,3071.6,3130.8,2978.4
4491,176.985751,50.563472,0,104,41,0,310.8,315.0,334.4,352.4,...,3411.8,3500.4,3447.0,3475.6,3441.2,3543.6,3544.6,3460.8,3442.4,3428.6
4492,180.485751,50.563472,0,105,41,0,336.8,294.6,334.0,308.8,...,4396.2,4269.8,4347.6,4419.6,4302.4,4475.6,4374.2,4383.8,4370.6,4268.2


## Funciones

In [18]:
class WhittakerSmoother(object):
    def __init__(self, signal, smoothness_param, deriv_order=1):
        self.y = signal
        assert deriv_order > 0, 'deriv_order must be an int > 0'
        # Compute the fixed derivative of identity (D).
        d = np.zeros(deriv_order*2 + 1, dtype=int)
        # d[deriv_order] = 1
        d = np.diff(d, n=
        deriv_order)
        n = self.y.shape[0]
        k = len(d)
        s = float(smoothness_param)
        # Here be dragons: essentially we're faking a big banded matrix D,
        # doing s * D.T.dot(D) with it, then taking the upper triangular bands.
        diag_sums = np.vstack([
            np.pad(s*np.cumsum(d[-i:]*d[:i]), ((k-i,0),), 'constant')
            for i in range(1, k+1)])
        upper_bands = np.tile(diag_sums[:,-1:], n)
        upper_bands[:,:k] = diag_sums
        for i,ds in enumerate(diag_sums):
            upper_bands[i,-i-1:] = ds[::-1][:i+1]
        self.upper_bands = upper_bands
    def smooth(self, w):
        foo = self.upper_bands.copy()
        foo[-1] += w  # last row is the diagonal
        return solveh_banded(foo, w * self.y, overwrite_ab=True, overwrite_b=True)

def als_baseline(intensities, asymmetry_param=0.0001, smoothness_param=1e4,
                 max_iters=20, conv_thresh=1e-6, verbose=False):
    """
    Applies the asymmetric least squares (ALS) method to fit the baseline.

    Parameters:
    - intensities (numpy.ndarray or pandas.Series): Numpy array or pandas Series representing the Raman intensities.
    - asymmetry_param (float): Asymmetry parameter for the fit.
    - smoothness_param (float): Smoothness parameter for the fit.
    - max_iters (int): Maximum number of iterations.
    - conv_thresh (float): Convergence threshold.
    - verbose (bool): Flag to print debugging information.

    Returns:
    - baseline (numpy.ndarray): Numpy array representing the fitted baseline.
    """
    if isinstance(intensities, pd.Series):
        intensities = intensities.values

    smoother = WhittakerSmoother(intensities, smoothness_param, deriv_order=2)
    p = asymmetry_param
    w = np.ones(intensities.shape[0])

    for i in range(max_iters):
        z = smoother.smooth(w)
        mask = intensities > z
        new_w = p * mask + (1 - p) * (~mask)
        conv = np.linalg.norm(new_w - w)

        if verbose:
            print(i + 1, conv)

        if conv < conv_thresh:
            break

        w = new_w
    else:
        # print('ALS did not converge in %d iterations' % max_iters)
        pass

    return z



def find_baseline(df):
    """
    Apply baseline correction to each row in a DataFrame.
    Input:
        df: DataFrame, contains the spectral data.
    Output:
        DataFrame, with baseline corrected for each spectrum.
    """
    D0_BL = df.copy()
    for i in range(len(df)):
        # Apply ALS baseline correction to each row
        bl = als_baseline(df.iloc[i, :].values)
        D0_BL.iloc[i, :] = bl
    return D0_BL

def remove_background(df):
    D0_BG = df.copy()

    return D0_BG

def correct_baseline(data, baseline):
    """
    Corrects the baseline of the data.
    Parameters:
    - data (pandas.DataFrame or pandas.Series): DataFrame or Series containing the data.
    - baseline (pandas.DataFrame or pandas.Series): DataFrame or Series with the baselines.
    Returns:
    - corrected_data (pandas.DataFrame): DataFrame with the corrected data.
    """
    # Ensure that both data and baseline are DataFrames
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)

    if not isinstance(baseline, pd.DataFrame):
        baseline = pd.DataFrame(baseline)

    # Perform baseline correction
    corrected_data = data.values - baseline.values

    # Convert the result back to a DataFrame while maintaining the same column names
    corrected_data = pd.DataFrame(corrected_data, columns=data.columns).copy()

    return corrected_data


def normalize_spectra(X_filtered):
    """
    Preprocessing of spectral data.
    Input:
        X_filtered: DataFrame, raw spectral data.
    Output:
        DataFrame, preprocessed spectral data.
    """
    X_norm_list = []
    for _, row in X_filtered.iterrows():
        # Normalize each spectrum by dividing by the sum and multiplying by a constant
        total = np.sum(row)
        X_norm_list.append(pd.DataFrame([row / total * 2000]))

    X_norm = pd.concat(X_norm_list, ignore_index=True)

    # Apply Savitzky-Golay filter for smoothing
    X_norm_flt = savgol_filter(X_norm, 21, 2)
    X_norm_flt = pd.DataFrame(X_norm_flt, columns=X_filtered.columns)

    '''# Perform baseline correction
    bl = find_baseline(X_norm_flt)
    X_norm_flt = X_norm_flt - bl'''

    # Standardize the data using StandardScaler
    scaler = StandardScaler()
    z_numpy = X_norm_flt.values
    z_numpy_scaled_rows = scaler.fit_transform(z_numpy.T).T
    X_norm_flt_stdz = pd.DataFrame(data=z_numpy_scaled_rows, columns=X_filtered.columns)

    return X_norm_flt_stdz



def procesarRamanNanobase(file):
    
    df = pd.read_csv(file, engine='python', sep=',', skiprows=14, decimal='.')
    resultados = []
    archivo= file

    # # Drop unwanted column
    if 'Unnamed: 1937' in df.columns:
            df = df.drop(columns=['Unnamed: 1937'])

    # Store coordinates
    coordenadas = df.iloc[:, 0:2].T
        
    # Select only ramanshift columns
    df2 = df.iloc[:, 7:]

    # Create a DataFrame for Raman shift values
    ramanshitfs = pd.DataFrame(df2.columns.astype(float), columns=['Raman shift'])

    # Select Raman spectra data within the desired range
    D = df2.loc[:, (df2.columns.astype(float) >= 500) & (df2.columns.astype(float) <=2000)]

    # Filter Raman shift values within the desired range
    ramanshitfs = ramanshitfs.loc[(ramanshitfs['Raman shift'] >= 500) & (ramanshitfs['Raman shift'] <= 2000)]


    #Filtrado de espectros saturados
    # guardado del número total de filas
    total_rows = len(D)
    # Descartar las filas donde cualquier valor supera 65000
    D = D[~(D > 65000).any(axis=1)]

    # Guardar el número de filas después de descartar
    remaining_rows = len(D)
    # Calcular e imprimir el número de filas descartadas
    discarded_rows = total_rows - remaining_rows
    print(f'{archivo}: se descartaron {discarded_rows} espectros saturados de un total de {total_rows} espectros.')
    
    # Find baseline of Raman spectra data
    D_BL = find_baseline(D)
    # Correct baseline of Raman spectra data
    D_BC =correct_baseline(D, D_BL)
    # Normalize Raman spectra data
    D_PP = normalize_spectra(D_BC)

    df_final = pd.concat([coordenadas.T, D_PP], axis=1)
    df_final.set_index(['X', 'Y'], inplace=True)

    tipo_laser = "Nanobase"
    nombre_archivo = archivo.split('\\')[-1].split('.')[0]
    nombre_muestra = nombre_archivo.split('_')[1]
    fecha_medida = nombre_archivo.split('_')[0]
    hora_medida = nombre_archivo.split("_")[0].split("-")[1]
    tipo = 'raw'
    df_final.index = pd.MultiIndex.from_tuples([( tipo_laser, nombre_muestra, tipo,
                                                        fecha_medida,hora_medida, nombre_archivo, X, Y, ) for X, Y in df_final.index],
                                                  names=[ "Instrumento","Muestra","Tipo",  "Fecha","Hora", "Archivo","X", "Y"])
    
    # Append processed data to the results list
    resultados.append((archivo, coordenadas, D, D_BL, D_PP, ramanshitfs))

    return df_final, resultados


def procesarRamanSonda(file):
    resultados_sonda =[]
    archivo = file

    df_sonda = pd.read_csv(file)
    # Store coordinates
    df_sonda_reset = df_sonda.reset_index()
    coordenadas_sonda = df_sonda_reset.iloc[:, :2]
    coordenadas_sonda.columns = ['X', 'Y']
    coordenadas_sonda = coordenadas_sonda.T
            
    df2_sonda = df_sonda.iloc[:, 3:]

    # Create a DataFrame for Raman shift values
    ramanshitfs_sonda = pd.DataFrame(df2_sonda.columns.astype(float), columns=['Raman shift'])
    # Select Raman spectra data within the desired range
    D_sonda = df2_sonda.loc[:, (df2_sonda.columns.astype(float) >= 500.0) & (df2_sonda.columns.astype(float) <= 2000.0)]
    # Filter Raman shift values within the desired range
    ramanshitfs_sonda = ramanshitfs_sonda.loc[(ramanshitfs_sonda['Raman shift'] >= 500) & (ramanshitfs_sonda['Raman shift'] <= 2000)]
    #Filtrado de espectros saturados
    total_rows_sonda = len(D_sonda)
    # Descartar las filas donde cualquier valor supera 65000
    D_sonda = D_sonda[~(D_sonda > 65000).any(axis=1)]
    # Guardar el número de filas después de descartar
    remaining_rows_sonda = len(D_sonda)
    # Calcular e imprimir el número de filas descartadas
    discarded_rows_sonda = total_rows_sonda - remaining_rows_sonda

    archivo_sonda ='Sonda'
    print(f'{archivo_sonda}: se descartaron {discarded_rows_sonda} espectros saturados de un total de {total_rows_sonda} espectros.')

        # Removing Background
    D_BG_sonda = remove_background(D_sonda)
    # Find baseline of Raman spectra data
    D_BL_sonda = find_baseline(D_BG_sonda)
    # Correct baseline of Raman spectra data
    D_BC_sonda = correct_baseline(D_sonda, D_BL_sonda)
    # Normalize Raman spectra data
    D_PP_sonda = normalize_spectra(D_BC_sonda)

    
    df_sonda_final = pd.concat([coordenadas_sonda.T, D_PP_sonda], axis=1)
    df_sonda_final.set_index(['X', 'Y'], inplace=True)

    resultados_sonda.append((archivo, coordenadas_sonda, D_sonda, D_BL_sonda, D_PP_sonda, ramanshitfs_sonda))

    nombre_archivo = archivo.split("\\")[-2] # Nombre del archivo
    nombre_muestra = nombre_archivo.split("_")[3] # Nombre de la muestra
    fecha_medida = nombre_archivo.split("_")[0].split("-")[0]  # Fecha de la medida
    hora_medida = nombre_archivo.split("_")[0].split("-")[1]    # Hora de la medida
    tipo_laser =   f"Sonda {nombre_archivo.split('_')[2]}"
    tipo = 'raw'
    df_sonda_final.index = pd.MultiIndex.from_tuples([( tipo_laser, nombre_muestra, tipo,
                                                        fecha_medida,hora_medida, nombre_archivo, X, Y, ) for X, Y in df_sonda_final.index],
                                                  names=[ "Instrumento","Muestra","Tipo",  "Fecha","Hora", "Archivo","X", "Y"])
    return df_sonda_final, resultados_sonda